## Assignment19:

1. Train any GAN to create 64x64 images of Indian Cars.
  Ref: https://github.com/simontomaskarlsson/CycleGAN-Keras
2. You have a team of 4 people, divide the project between yourselves including image collection [1000 Images], model selection, etc. 
3. The deadline ends next to next week.

In [0]:
!wget 'https://github.com/arjuntheprogrammer/TheSchoolOfAI/raw/master/Session19/carImages.zip'

In [0]:
from zipfile import ZipFile 
file_name = "/content/carImages.zip"
with ZipFile(file_name, 'r') as zip: 
    zip.extractall() 
    print('Done!') 

Done!


In [0]:
import cv2
import os 
import shutil

def resize_images(input_image_path, output_image_path):
  ori_img = cv2.imread(input_image_path,cv2.IMREAD_COLOR)
  try:
    new_img = cv2.resize(ori_img,(int(64),int(64)))
    cv2.imwrite(output_image_path,new_img)
  except:
    print("image skipped")
  

In [0]:
i = 0
src_path = '/content/carImages/'
dst_path = '/content/carImagesResize/'

if os.path.exists(dst_path):
  shutil.rmtree(dst_path)
shutil.rmtree(dst_path)
if not os.path.exists(dst_path):
    os.makedirs(dst_path)
for filename in os.listdir(src_path):
  filename, file_extension = os.path.splitext(filename) 
  src = src_path + filename + file_extension 
  dst = dst_path + "img_" + str(i) + file_extension
  resize_images(src, dst)
  # os.rename(src, dst) 
  i += 1

image skipped
image skipped
image skipped


In [0]:
print(len((os.listdir(src_path))))
print(len((os.listdir(dst_path))))
for filename in (os.listdir(dst_path))[:5]:
  ori_img = cv2.imread(dst_path + filename,cv2.IMREAD_COLOR)
  print(ori_img.shape)

1437
1434
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)


In [0]:
from __future__ import print_function, division
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import numpy as np

In [0]:
def getXTrain():
  X_train = []
  for filename in (os.listdir(dst_path)):
    image = cv2.imread(dst_path + filename,cv2.IMREAD_COLOR)
    X_train.append(image)
  return np.array(X_train) 

In [0]:
class GAN():
    def __init__(self):
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, 1, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        # (X_train, _), (_, _) = mnist.load_data()
        X_train = getXTrain()
        print(X_train.shape)
        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # print("imgs.shape = ", imgs.shape)
            # print(imgs[0])

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                # Plot the progress
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                
                # save the genImages
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("/content/carImagesGen/%d.png" % epoch)
        plt.close()

In [0]:
gen_path = "/content/carImagesGen/"
if os.path.exists(gen_path):
  shutil.rmtree(gen_path)
if not os.path.exists(gen_path):
    os.makedirs(gen_path)

gan = GAN()
gan.train(epochs=30000, batch_size=32, sample_interval=200)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 12288)             0         
_________________________________________________________________
dense_57 (Dense)             (None, 512)               6291968   
_________________________________________________________________
leaky_re_lu_41 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_58 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_42 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 1)                 257       
Total params: 6,423,553
Trainable params: 6,423,553
Non-trainable params: 0
___________________________________________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.690159, acc.: 39.06%] [G loss: 0.635221]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


200 [D loss: 0.411243, acc.: 84.38%] [G loss: 1.908783]
400 [D loss: 0.543537, acc.: 67.19%] [G loss: 1.416030]
600 [D loss: 0.889799, acc.: 50.00%] [G loss: 1.531708]
800 [D loss: 0.579543, acc.: 68.75%] [G loss: 1.510026]
1000 [D loss: 0.582561, acc.: 70.31%] [G loss: 1.718539]
1200 [D loss: 0.403486, acc.: 84.38%] [G loss: 2.114439]
1400 [D loss: 0.329018, acc.: 84.38%] [G loss: 2.524767]
1600 [D loss: 0.345726, acc.: 82.81%] [G loss: 2.653835]
1800 [D loss: 0.253199, acc.: 92.19%] [G loss: 3.033704]
2000 [D loss: 0.236076, acc.: 92.19%] [G loss: 2.920671]
2200 [D loss: 0.284030, acc.: 84.38%] [G loss: 3.892900]
2400 [D loss: 0.350718, acc.: 79.69%] [G loss: 2.879710]
2600 [D loss: 0.197352, acc.: 92.19%] [G loss: 5.264869]
2800 [D loss: 0.171306, acc.: 93.75%] [G loss: 3.332701]
3000 [D loss: 0.144110, acc.: 93.75%] [G loss: 3.952491]
3200 [D loss: 0.206475, acc.: 90.62%] [G loss: 3.866613]
3400 [D loss: 0.233926, acc.: 92.19%] [G loss: 4.212934]
3600 [D loss: 0.372798, acc.: 87.50